In [17]:
import os
import subprocess
import tempfile
import requests
import json


# Set up your Groq API key
GROQ_API_KEY = "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"

# Define the Groq API endpoint and headers
GROQ_API_ENDPOINT = 'https://api.groq.com/openai/v1/chat/completions'  # Replace with the actual endpoint
HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {GROQ_API_KEY}',
}

def groq_chat_completion(messages, model='llama3-70b-8192', temperature=0):
    payload = {
        'model': model,
        'messages': messages,
        'temperature': temperature,
        # Include other necessary parameters as per Groq API documentation
    }

    response = requests.post(
        GROQ_API_ENDPOINT,
        headers=HEADERS,
        data=json.dumps(payload)
    )

    if response.status_code == 200:
        result = response.json()
        # Adjust the parsing based on Groq's response format
        content = result['choices'][0]['message']['content']
        return content.strip()
    else:
        raise Exception(f"Groq API error: {response.status_code} - {response.text}")






def test_agent(user_query, code, test_output, test_errors):
    system_message = {
        "role": "system",
        "content": "You are a test agent. Evaluate the programmer's code based on the test results and provide feedback."
    }
    user_message = {
        "role": "user",
        "content": f"""
The programmer wrote the following code:

{code}

The tests produced the following output:

{test_output}

There were errors:

{test_errors}

Provide an explanation of the errors and suggest how to fix them.
"""
    }

    messages = [system_message, user_message]

    explanation = groq_chat_completion(messages)
    return explanation





In [18]:
def clean_code(code):
    """
    Clean the code by removing unwanted markdown markers or other invalid syntax.
    This will ensure the code is valid Python code.
    """
    # Remove Markdown code block markers if present
    code = code.replace('```', '')
    return code.strip()

def programmer_agent(user_query):
    system_message = {
        "role": "system",
        "content": "You are a programmer. Write Python code to solve the user's problem. Provide only the code, do not include explanations."
    }
    user_message = {
        "role": "user",
        "content": f"Problem:\n\n{user_query}\n\nEncapsulate your solution in functions or classes as appropriate."
    }

    messages = [system_message, user_message]

    code = groq_chat_completion(messages)
    return clean_code(code)  # Clean the code before returning

def test_designer_agent(user_query):
    system_message = {
        "role": "system",
        "content": "You are a test designer. Write relevant Python unit tests to verify the correctness of code solving the user's problem. Use the unittest framework in Python. Provide only the test code, do not include explanations."
    }
    user_message = {
        "role": "user",
        "content": f"Problem:\n\n{user_query}\n\nAssume the programmer's code is in a file named 'code.py'. Import the necessary functions or classes from 'code.py' to perform the tests."
    }

    messages = [system_message, user_message]

    test_code = groq_chat_completion(messages)
    return clean_code(test_code)  # Clean the test code before returning

# Example: Adding this clean_code function to remove unwanted markdown markers
def code_executor(code, test_code):
    with tempfile.TemporaryDirectory() as tempdir:
        # Write the programmer's code to code.py
        code_file = os.path.join(tempdir, 'code.py')
        with open(code_file, 'w') as f:
            f.write(code)

        # Write the test code to test_code.py
        test_file = os.path.join(tempdir, 'test_code.py')
        with open(test_file, 'w') as f:
            f.write(test_code)

        # Run the tests using unittest
        try:
            result = subprocess.run(
                ['python', '-m', 'unittest', 'test_code.py'],
                cwd=tempdir,
                capture_output=True,
                text=True,
                check=True
            )
            output = result.stdout
            errors = result.stderr
            return True, output, errors
        except subprocess.CalledProcessError as e:
            # Tests failed
            output = e.stdout
            errors = e.stderr
            return False, output, errors


In [20]:
user_query = input("Enter your query: ")

# Programmer writes code
code = programmer_agent(user_query)
print("\nProgrammer's code:")
print(code)

# Test designer writes tests
test_code = test_designer_agent(user_query)
print("\nTest Designer's test code:")
print(test_code)

# Execute code and tests
success, output, errors = code_executor(code, test_code)

if success:
    print("\nAll tests passed!")
else:
    print("\nTests failed.")
    print("Test Output:")
    print(output)
    print("Test Errors:")
    print(errors)

    # Test agent evaluates the code
    explanation = test_agent(user_query, code, output, errors)
    print("\nTest Agent's feedback:")
    print(explanation)



Programmer's code:
def add_numbers(a, b):
    return a + b

result = add_numbers(1, 2)
print(result)

Test Designer's test code:
import unittest
from code import calculate_expression

class TestCalculateExpression(unittest.TestCase):

    def test_calculate_expression(self):
        self.assertEqual(calculate_expression("1+2"), 3)

if __name__ == '__main__':
    unittest.main()

Tests failed.
Test Output:
3

Test Errors:
E
ERROR: test_code (unittest.loader._FailedTest.test_code)
----------------------------------------------------------------------
ImportError: Failed to import test module: test_code
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/unittest/loader.py", line 162, in loadTestsFromName
    module = __import__(module_name)
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/private/var/folders/33/2qsqr9pj7r9bm6r8m0hvw9yr0000gn/T/tmp1pwxnppv/test_code.py", line 2, in <module>
    from code import calculate_expression
ImportError: cannot import name 'calcu